# Load data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Setup the initial weights for training (If not, the weights for energy and force will be both equal to 1)

In [2]:
import numpy as np
from mlearn.data import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces)
_, df = convert_docs(train_pool)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 100

# Set up the qSNAP and train

In [3]:
from mlearn.describers import BispectrumCoefficients
from mlearn.models import LinearModel
from mlearn.potentials.snap import SNAPotential

element_profile = {'Mo': {'r': 0.5, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=5.0, twojmax=4, element_profile=element_profile, 
                                   quadratic=True, pot_fit=True)
model = LinearModel(describer=describer)
qsnap = SNAPotential(model=model)
qsnap.train(train_structures, train_energies, train_forces, weights=weights)

# Lattice constant, Elastic constant

In [4]:
from mlearn.potentials.lammps.calcs import LatticeConstant

conventional_cell = Structure.from_file('conventional.cif')
lc_calculator = LatticeConstant(ff_settings=qsnap)
a, b, c = lc_calculator.calculate([conventional_cell])[0]
print('Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Lattice a: 3.04335655890872, Lattice b: 3.04335655890872, Lattice c: 3.04335655890872


In [5]:
from mlearn.potentials.lammps.calcs import ElasticConstant

ec_calculator = ElasticConstant(ff_settings=qsnap, lattice='bcc', alat=3.106)
C11, C12, C44, bulk_modulus = ec_calculator.calculate()
print('C11: {}, C12: {}, C44: {}, bulk modulus: {}'.format(C11, C12, C44, bulk_modulus))

C11: 3227.02362265649, C12: 2483.42898841636, C44: 436.179050004324, bulk modulus: 2731.29386648626


# Load model from parameters files

In [6]:
from mlearn.potentials.snap import SNAPotential

qsnap_loaded = SNAPotential.from_config(param_file='SNAPotential.snapparam', coeff_file='SNAPotential.snapcoeff')

# Energy, force, stress prediction

In [7]:
from mlearn.potentials.lammps.calcs import EnergyForceStress

struct = Structure.from_file('test_struct.cif')
efs_calculator = EnergyForceStress(ff_settings=qsnap_loaded)
energy, forces, stresses = efs_calculator.calculate([struct])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

energy: -463.40729428822
forces: 
 [[ -4.35009     1.97057     6.87678  ]
 [ -5.84681     4.50577     2.92247  ]
 [  4.84572    -9.812       6.3182   ]
 [  3.26106    -1.27683     2.03582  ]
 [ -0.745435    1.46983    -7.68259  ]
 [  0.104191   -9.01277    12.0424   ]
 [ 11.4797     -8.32839    -4.25459  ]
 [ -3.63831     3.00988     6.97344  ]
 [  5.67339     3.77283     3.84525  ]
 [ -4.45253     8.4348     15.8105   ]
 [  8.3741     -4.00962     5.70634  ]
 [  7.81246     3.48492    -3.91555  ]
 [  5.17492    -2.41186    -4.50414  ]
 [ -7.8016      3.79974    -0.286845 ]
 [-16.0136      1.20342     0.508142 ]
 [-11.224       3.02393    10.3126   ]
 [ -9.51594    -9.97033    -6.72337  ]
 [  2.60513    -2.00806     2.40102  ]
 [ -3.34591    -2.32787     4.79208  ]
 [-16.8236      1.90041    -0.96854  ]
 [  0.410324   -8.81339     2.46136  ]
 [-16.1998     -7.42978    -9.55334  ]
 [ -0.168133   -3.04826    -3.7939   ]
 [  3.89159    -3.70051    -3.79918  ]
 [ 14.7243     -2.7781     -4